In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd ../content/drive/MyDrive/Wanted/SUM

/content/drive/MyDrive/Wanted/SUM


In [ ]:
!pip install html2text

In [ ]:
import pandas as pd
import pickle
import requests
import json
import time
import re
from html2text import html2text
from tqdm.notebook import tqdm

# Load Original Data

In [ ]:
df = pd.read_csv("./data/sports_news_data.csv")
df = df.dropna()
df = df.reset_index(drop=True)
len(df)

9075

In [ ]:
df.head()

,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15


# Execute CLOVA Summary API

- CLOVA에서 제공하는 문서요약 API를 사용하여 각 기사 원문에 대한 정답 레이블 생성

In [ ]:
# from au_tokens import API_ID, API_SECRET
# *au_tokens.py
API_ID = '6h2130teeb'
API_SECRET = 'eeSUFwRMav4lXyQXN8WQHuK3yyCTEVFVQ88l80P0'

In [ ]:
# https://api.ncloud-docs.com/docs/ai-naver-clovasummary-api
def summary(content, title):
    
    url = "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize"

    # Authorization 
    clientId = API_ID
    clientSecret = API_SECRET

    headers = {
        "X-NCP-APIGW-API-KEY-ID": clientId,
        "X-NCP-APIGW-API-KEY": clientSecret,
        "Content-Type": "application/json"
    }
    
    # document
    content = content
    title = title
    
    # option
    language = "ko"
    model = "general"
    tone = 0
    summaryCount = 3 # Summary의 총 문장 개수를 3개로 설정


    data = {
        "document": {
            "content": content,
            "title": title
        },
        "option": {
            "language": language,
            "model": model,
            "tone": tone,
            "summaryCount": summaryCount
        }
    }

    response = requests.post(url, data=json.dumps(data), headers=headers, timeout=30)
    rescode = response.status_code

#     if (rescode == 200):
#         print(response.text)
#     else:
#         print("Error : " + response.text)

    result = html2text(response.text[12:-2])

    return result

# 분담하여 API 돌린 결과물 불러오기

In [ ]:
with open('label06000.txt', 'rb') as f_in:
    data_first_labels = pickle.load(f_in)

data_second_df = pd.read_csv('clova_data_from6000.csv')
data_second_labels = data_second_df['LABEL'].values
data_labels = data_first_labels + list(data_second_labels)
print(len(data_labels))

9075


In [ ]:
df['LABEL'] = data_labels
df.head()

,TITLE,CONTENT,PUBLISH_DT,LABEL
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15,스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 ...
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n영국 매체 '스포츠...
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15,스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다...
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15,바르사는 15일(한국 시간) 공식 홈페이지를 통해 2021/2022시즌에 입을 어웨...


In [ ]:
# Save Data

df.to_csv('clova_summary_data_total.csv', index=False)

# Data Preprocessing

In [ ]:
df = pd.read_csv('clova_summary_data_total.csv')

### 1. LABEL(정답 요약문)이 문장 3개로 이루어져 있는지 확인

In [ ]:
# Add Additional Columns

df['LABEL_CNT'] = df['LABEL'].apply(lambda x: len(x.split('\\n')))
df.head()

,TITLE,CONTENT,PUBLISH_DT,LABEL,LABEL_CNT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...,3
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15,스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 ...,3
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n영국 매체 '스포츠...,3
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15,스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다...,3
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15,바르사는 15일(한국 시간) 공식 홈페이지를 통해 2021/2022시즌에 입을 어웨...,3


In [ ]:
df['LABEL_CNT'].value_counts() # API의 결과로 나온 summary 내용 중 3문장으로 구성되어 있지 않은 데이터가 존재

3    8389
1     405
2     281
Name: LABEL_CNT, dtype: int64

In [ ]:
# API의 오류로 인해 LABEL이 제대로 뽑히지 않은 경우, '\\n' 구분자로 문장이 제대로 분리되어 나오지 않은 경우가 이에 해당함
df[df['LABEL_CNT'] != 3].tail()

,TITLE,CONTENT,PUBLISH_DT,LABEL,LABEL_CNT
9003,"'같은 실수 되풀이 안 한다' 리버풀, 이적 마감날 영입무산→재협상 예정",<p>[골닷컴] 강동훈 기자 = 리버풀이 이번 시즌이 끝나면 파비우 카르발류(19·...,2022-02-06,리버풀이 이번 시즌이 끝나면 파비우 카르발류(19·풀럼)를 자유계약(FA)으로 영입...,2
9004,"[GOAL 리뷰] '미나미노 결승포' 리버풀, 카디프에 3-1 승. FA컵 16강행",<p>[골닷컴] 박문수 기자 = 리버풀이 카디프 시티와의 FA컵 4라운드(32강) ...,2022-02-06,"케이타의 패스 그리고 조타의 절묘한 드리블이 돋보였지만, 필립스가 막아냈다.\n\n...",1
9008,'자동문&최다 실점' 보르도.. '3경기 15실점' 황의조 혼자 되겠어?,<p>[골닷컴] 박문수 기자 = 황의조 소속팀 보르도가 결국 강등권으로 밀려났다. ...,2022-02-06,"0,""error"":{""errorCode"":""E003"",""message"":""Text ...",1
9009,"[GOAL 리뷰] '황의조 무득점' 보르도, 랭스에 0-5 대패. 강등권 추락",<p>[골닷컴] 박문수 기자 = 황의조가 선발 출전한 지롱댕 보르도가 스타드 랭스 ...,2022-02-06,"0,""error"":{""errorCode"":""E003"",""message"":""Text ...",1
9055,"[김병윤의 축구생각] 선수 생활은 짧다, 제2의 축구 인생을 준비 하라",[스포탈코리아]축구에 처음으로 입문하는 대부분 선수에게 공통점이 있다. 그것은 선수...,2022-02-07,"0,""error"":{""errorCode"":""E003"",""message"":""Text ...",1


### 2. CONTENT(뉴스기사), LABEL(요약문)에서 특수기호 등 제거

In [ ]:
import re

def remove_str(sent):
    sent = sent.replace('\n\n', ' ')
    sent = sent.replace('\n', ' ')
    sent = sent.replace('\\n', '\t')
    sent = sent.replace('\\"', '"')
    sent = sent.replace("\\'", "'")
    sent = sent.replace("’", "'") # 서로 다른 홑따옴표 모양 통일
    sent = sent.replace("‘", "'") # 서로 다른 홑따옴표 모양 통일
    sent = sent.replace("“", '"') # 서로 다른 쌍따옴표 모양 통일
    sent = sent.replace('”', '"') # 서로 다른 쌍따옴표 모양 통일
    sent = sent.replace(',', ',')
    sent = sent.replace(r'*', '')
    sent = sent.replace('<br />', '')
    sent = sent.replace(r'&#160;', ' ').replace(r'&#8211;', ' ').replace(r'&#65279;', '')
    sent = sent.replace('스포탈코리아]', '[스포탈코리아]')
    sent = re.sub('[|-]', '', sent)
    sent = re.sub(r'[[가-힣=\(\)]+]', '', sent) # [스포탈코리아], [골닷컴] 등 헤더 제거
    sent = re.sub('좌절됐다, ?이날 경기', '좌절됐다. 이날 경기', sent)
    sent = re.sub('\(\\\\"https\:[\.0-9A-Za-z%\/]+\\\\"\)', '', sent) # url 정보 제거
    sent = re.sub('주요 뉴스.+zip""', '', sent)
    sent = re.sub('주요 뉴스.+blank">"', '', sent)
    sent = re.sub('주요 뉴스', '', sent)
    sent = re.sub('<img src=[^가-힣]+\">', '', sent)
    sent = re.sub('<source.+height\:[0-9]{3}\">', '', sent)
    sent = sent.replace(r'[" 토트넘 선수들의 연애 전선은?"]', '')
    sent = sent.replace('토트넘 선수들의 연애 전선은?"', '')
    sent = re.sub('</?[a-zA-Z]+>', ' ', sent)
    sent = re.sub('[가-힣]{3} (기자|통신원) ?=', '', sent) # 기자 정보 제거
    sent = re.sub('사진=[0-9a-zA-Z가-힣]+', '', sent) # 사진 관련 캡션 제거
    sent = re.sub(' +', ' ', sent) # 띄어쓰기 2개 이상 -> 하나로 수정
    sent = sent.replace('" "', '""')
    sent = sent.replace('..', '.')
    sent = re.sub(r"([가-힣\)])\.([0-9가-힣])", r"\1. \2", sent)
    sent = re.sub(r"([가-힣\)])\?([0-9가-힣])", r"\1? \2", sent)
    sent = re.sub(r"([가-힣\)]),([0-9가-힣])", r"\1, \2", sent)
    sent = sent.replace(r'zip"" ', '')
    sent = sent.replace(r'\ ', '')
    sent = sent.strip()

    return sent

In [ ]:
df['CONTENT'] = df['CONTENT'].apply(remove_str)
df['LABEL'] = df['LABEL'].apply(remove_str)

In [ ]:
df = df[['TITLE', 'CONTENT', 'PUBLISH_DT', 'LABEL']]
df.head()

,TITLE,CONTENT,PUBLISH_DT,LABEL
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...,2021-07-15,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까. 지난...,2021-07-15,유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.\t공...
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다. 영국 매체 '스포츠 ...,2021-07-15,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\t영국 매체 '스포츠...
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다. 이재성은 2...,2021-07-15,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다.\t1995년...
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽다는 반응이다. ...,2021-07-15,바르사는 15일(한국 시간) 공식 홈페이지를 통해 2021/2022시즌에 입을 어웨...


### 3. 문장 구분자로 '\t' 추가해주기

In [ ]:
# '까. 다. 까? 등 기준으로 문장이 나뉘어진다고 간주하고, 문장 구분자로 '\t' 넣어주기

def add_tab_btw_sentences(sent):
    sent = sent.replace('. ', '.\t') # content용
    sent = sent.replace('? ', '?\t')
    sent = sent.replace('다." ', '다."\t')
    sent = sent.replace('." ', '."\t')
    sent = sent.replace(".'", ".'\t")
    return sent

In [ ]:
tab_df = df.copy()
tab_df['CONTENT'] = tab_df['CONTENT'].apply(add_tab_btw_sentences)
tab_df['LABEL'] = tab_df['LABEL'].apply(add_tab_btw_sentences)
tab_df.head()

,TITLE,CONTENT,PUBLISH_DT,LABEL
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...,2021-07-15,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.\t지...,2021-07-15,유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.\t공...
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\t영국 매체 '스포츠...,2021-07-15,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\t영국 매체 '스포츠...
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다.\t이재성은 ...,2021-07-15,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다.\t1995년...
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다.\t팬들은 만족스럽다는 반응이다....,2021-07-15,바르사는 15일(한국 시간) 공식 홈페이지를 통해 2021/2022시즌에 입을 어웨...


In [ ]:
tab_df.to_csv('sports_news_with_tab.csv', index=False)

### 4. LABEL(요약문)의 문장 개수 재확인

In [ ]:
# 탭('\t') 기준으로 문장을 split 했을 때 summary 내용(LABEL)의 최종 문장 개수
# Summary 문장은 총 3개여야 함

tab_df = pd.read_csv('sports_news_with_tab.csv')
tab_df['LABEL_CNT'] = tab_df['LABEL'].apply(lambda x:len(x.split('\t')))
tab_df[['LABEL_CNT']].value_counts() # API 결과물에서 문장으로 제대로 구분되지 않았던 것들이 문장으로 구분되며 summary의 문장 개수가 4개 이상인 데이터도 보임

LABEL_CNT
3            7126
5             567
4             556
6             427
1             354
2              22
7              22
8               1
dtype: int64

In [ ]:
# Summary 내용(LABEL)의 최종 문장 개수가 3개가 아닌 데이터의 경우, LABEL값 None으로 대체

not_3_idx = tab_df[tab_df['LABEL_CNT'] != 3].index

for idx in not_3_idx:
    tab_df.loc[idx, 'LABEL'] = None

new_label_cnt = []

for i in range(len(tab_df)):
    if tab_df.iloc[i]['LABEL'] is not None:
        new_label_cnt.append(len(tab_df.iloc[i]['LABEL'].split('\t')))
    else:
        new_label_cnt.append(0) # LABEL이 None으로 교체된 데이터의 경우, LABEL_CNT를 0으로 교체

# LABEL_CNT Check
tab_df['LABEL_CNT'] = new_label_cnt
tab_df[['LABEL_CNT']].value_counts()

LABEL_CNT
3            7126
0            1949
dtype: int64

### 5. 문장 구분자 '\t'로 나뉘어진 CONTENT(뉴스기사), LABEL(요약문) 각 문장의 형태 통일

- 이후에 각 요약 문장이 뉴스기사 문장 중에서 몇 번째 인덱스에 위치해 있는지의 정보를 추출하기 위한 과정

In [ ]:
not_match_idx = []

for i in tqdm(range(len(tab_df))):
    data_point = tab_df.loc[i]

    if tab_df.loc[i]['LABEL'] is not None:
        
        doc_lst = list(map(lambda x: x.strip(), data_point['CONTENT'].split('\t'))) # 뉴스기사를 문장 단위로 분리한 리스트
        summary_lst = list(map(lambda x: x.strip(), data_point['LABEL'].split('\t'))) # 요약문을 문장 단위로 분리한 리스트

        # summary_lst의 요약 문장 하나씩을 뉴스기사 문장과 대조
        # 포맷이 일치하지 않는 요약 문장이 있는 경우, not_match_idx에 인덱스 정보 저장

        for sent in summary_lst:
            try:
                idx = doc_lst.index(sent)
                # print('SENT FOUND (SUCCESS)')
                # print(sent)
                # print('---DOC LIST---')
                # for doc in doc_lst:
                #     print(doc)
                # print()
            except:
                not_match_idx.append(i)
                print(f'SENT NOT FOUND, Data Point idx: {i}')
                print(sent)
                print('---DOC LIST---')
                for doc in doc_lst:
                    print(doc)
                print()

print(len(not_match_idx))

  0%|          | 0/9075 [00:00<?, ?it/s]

SENT NOT FOUND, Data Point idx: 120
메시는 우승 직후 코파 아메리카 우승컵을 껴안은 자신의 사진을 인스타그램에 올렸고 "정말 믿을 수 없다!
---DOC LIST---
리오넬 메시가 크리스티아누 호날두(유벤투스)를 제치고 스포츠인 중 인스타그램 게시글 최다 '좋아요'를 받았다.
영국 매체 '90min'은 18일(현지시간) "메시가 올린 코파 아메리카 우승 사진이 호날두를 제치고 가장 많은 좋아요를 받은 스포츠 게시물이 됐다"라고 전했다.
메시는 지난 2021 코파 아메리카에서 감격적인 우승을 이끌었다.
본인의 아르헨티나 국가대표팀 메이저 첫 우승이었다.
메시는 바르셀로나에서 유럽축구연맹(UEFA) 챔피언스리그, 프리메라리가, 코파 델 레이 등 여러 대회에서 정상에 섰지만 유독 대표팀 성적과는 거리가 멀었다.
월드컵 준우승을 비롯해 여러 차례 코파 아메리카 준우승에 그치며 고개를 떨궜다.
불혹의 나이에 접어든 메시는 이번 대회가 사실상 우승할 수 있는 마지막 기회였다.
우승에 대한 의지가 어느 때보다 컸던 메시는 원맨쇼를 펼치며 아르헨티나를 결승까지 견인했고 브라질와의 결승전에서 헌신한 끝에 우승컵을 들어올릴 수 있었다.
메시는 우승 직후 코파 아메리카 우승컵을 껴안은 자신의 사진을 인스타그램에 올렸고 "정말 믿을 수 없다! 감사합니다.
하느님! 아르헨티나가 챔피언이에요!"라며 기쁨을 감추지 못했다.
그리고 이 게시글은 무려 2천만이 넘는 좋아요를 끌어모으며 SNS에서 가장 인기 있는 스포츠 게시글이 됐다.
종전 최다 좋아요는 호날두의 디에고 마라도나 추모 사진이었는데 2천만을 넘지 못했다.
한편, 메시는 바르셀로나와 재계약을 앞두고 있다.
5년 재계약에 연봉을 절반이나 삭감한 것으로 알려졌다.
인스타그램

SENT NOT FOUND, Data Point idx: 187
FC바르셀로나가 중국 시장을 고려해 스웨덴 스파 브랜드 'H&M;'과 협상을 중단했다.
---DOC LIST---
FC바르셀로나가 중국 시장을 고려해 스웨덴 스파 브

In [ ]:
not_match_idx = list(set(not_match_idx))
print(len(not_match_idx))

115


In [ ]:
# 토큰 단위로 두 문장 사이의 유사도를 구해주는 함수

def return_similarity(a, b):
    '''
    a, b는 각각 문장을 구성하는 토큰들로 이루어져 있는 집합(set)
    '''
    c = a.intersection(b)
    return float(len(c))/(len(a)+len(b)-len(c))

- 기사 본문과 일치하지 않는 요약 문장이 있는 경우, 기사 본문의 문장 중에서 가장 유사도가 높은 문장으로 요약문을 치환

In [ ]:
mod_data_dict = dict()

for idx in tqdm(not_match_idx):
    data_point = tab_df.loc[idx]
    doc_lst = list(map(lambda x: x.strip(), data_point['CONTENT'].split('\t')))
    summary_lst = list(map(lambda x: x.strip(), data_point['LABEL'].split('\t')))

    mod_summary = []
    for summary in summary_lst:

        if summary in doc_lst:
            mod_summary.append(summary)
        
        else:
            max_doc, max_sim = '', -1000
            for doc in doc_lst:
                doc_sum_sim = return_similarity(set(doc.split()), set(summary.split()))
                if doc_sum_sim > max_sim:
                    max_sim = doc_sum_sim
                    max_doc = doc
            mod_summary.append(max_doc)
    
    if len(mod_summary) != 3:
        print(idx, doc_lst, summary_lst)

    mod_data_dict[idx] = '\t'.join(mod_summary)

  0%|          | 0/115 [00:00<?, ?it/s]

In [ ]:
len(mod_data_dict)

115

In [ ]:
for idx in tqdm(mod_data_dict.keys()):
    tab_df.loc[idx, 'LABEL'] = mod_data_dict[idx]

  0%|          | 0/115 [00:00<?, ?it/s]

In [ ]:
# 요약 문장과 매칭되지 않는 기사 원문이 있는지 마지막으로 체크
# len()결과가 0이라면 요약 문장이 모두 기사 본문과 일치한다는 뜻

not_match_idx = []

for i in tqdm(range(len(tab_df))):
    data_point = tab_df.loc[i]

    if tab_df.loc[i]['LABEL'] is not None:

        doc_lst = list(map(lambda x: x.strip(), data_point['CONTENT'].split('\t')))
        summary_lst = list(map(lambda x: x.strip(), data_point['LABEL'].split('\t')))

        for sent in summary_lst:
            try:
                idx = doc_lst.index(sent)
            except:
                not_match_idx.append(i)

print(len(not_match_idx))

  0%|          | 0/9075 [00:00<?, ?it/s]

0


In [ ]:
# Summary 문장의 위치 인덱스 정보 새로운 컬럼으로 추가

summary_idx_column_info = []
summary_idx_onehot_column_info = []

for i in tqdm(range(len(tab_df))):
    data_point = tab_df.loc[i]

    if data_point['LABEL'] is not None:

        doc_lst = list(map(lambda x: x.strip(), data_point['CONTENT'].split('\t')))
        summary_lst = list(map(lambda x: x.strip(), data_point['LABEL'].split('\t')))

        summary_idx = []
        onehot_idx = [0] * len(doc_lst)

        if len(summary_lst) != 3:
            print(doc_lst, summary_lst)

        for sent in summary_lst:
            try:
                idx = doc_lst.index(sent)
                summary_idx.append(idx)

            except:
                print(f'SENT NOT FOUND, Data Point idx: {i}')
                print(sent)
                print('---DOC LIST---')
                for doc in doc_lst:
                    print(doc)
                print()

        for idx in summary_idx:
            onehot_idx[idx] = 1

    else:
        summary_idx = None
        onehot_idx = None

    summary_idx_column_info.append(summary_idx)
    summary_idx_onehot_column_info.append(onehot_idx)

print(len(summary_idx_column_info), len(summary_idx_onehot_column_info))

  0%|          | 0/9075 [00:00<?, ?it/s]

9075 9075


In [ ]:
tab_df['SUMMARY_IDX'] = summary_idx_column_info
tab_df['SUMMARY_ONEHOT_IDX'] = summary_idx_onehot_column_info
tab_df.head(10)

,TITLE,CONTENT,PUBLISH_DT,LABEL,LABEL_CNT,SUMMARY_IDX,SUMMARY_ONEHOT_IDX
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...,2021-07-15,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...,3,"[0, 9, 11]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ..."
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.\t지...,2021-07-15,유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.\t공...,3,"[0, 8, 12]","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\t영국 매체 '스포츠...,2021-07-15,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\t영국 매체 '스포츠...,3,"[0, 1, 8]","[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다.\t이재성은 ...,2021-07-15,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다.\t1995년...,3,"[0, 21, 25]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다.\t팬들은 만족스럽다는 반응이다....,2021-07-15,바르사는 15일(한국 시간) 공식 홈페이지를 통해 2021/2022시즌에 입을 어웨...,3,"[2, 3, 12]","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
5,"긴급 수혈된 바르사 NO.9, 1년 반 만에 떠난다… ‘EPL행 유력’",FC 바르셀로나는 새 시즌을 앞두고 선수단 정리가 한창이다.\t잉여 자원들은 과감하...,2021-07-15,"스페인 매체 '스포르트'는 15일(한국 시간) ""바르사가 1,500만 유로(약 20...",3,"[3, 5, 15]","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,"[김남구의 유럽통신] 황의조, 손흥민 소속사와 손잡다… CAA Base와 계약",황의조(지롱댕 드 보르도)가 한국 선수로는 3번째로 세계적인 스포츠 에이전트사 CA...,2021-07-15,황의조(지롱댕 드 보르도)가 한국 선수로는 3번째로 세계적인 스포츠 에이전트사 CA...,3,"[0, 1, 2]","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
7,"""메시 종신은 축복!""…스폰서 5년 더 보장, 바르셀로나 함박웃음",리오넬 메시(34)가 FC바르셀로나에 남는다.\t연봉을 절반 삭감하지만 5년 계약으...,2021-07-15,영국 공영방송 'BBC'에 따르면 메시는 오는 2026년까지 바르셀로나와 5년 계약...,3,"[2, 3, 9]","[0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
8,"[오피셜] 눈물 흘렸던 '37세 전설' 로번, 두 번째 현역 은퇴 발표",네덜란드 축구스타 아르연 로번(37)이 현역 은퇴를 밝혔다.\t로번은 15일(한국시...,2021-07-15,네덜란드 축구스타 아르연 로번(37)이 현역 은퇴를 밝혔다.\t축구를 향한 사랑과 ...,3,"[0, 14, 15]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
9,"'100세' 메시팬 할아버지, 748골 수기 작성…메시도 감사 인사",리오넬 메시(34)는 프로 데뷔하고 748골을 터뜨렸다.\t전산화 하기에도 오랜 시...,2021-07-15,전산화 하기에도 오랜 시간이 걸리는 메시의 득점 기록을 일일이 손으로 작성한 팬이 ...,3,"[1, 4, 8]","[0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
# LABEL 문장이 3개로 구성되어 있는지 최종적으로 확인

new_label_cnt = []

for i in range(len(tab_df)):
    if tab_df.iloc[i]['LABEL'] is not None:
        new_label_cnt.append(len(tab_df.iloc[i]['LABEL'].split('\t')))
    else:
        new_label_cnt.append(0) # LABEL이 None으로 교체된 데이터의 경우, LABEL_CNT를 0으로 교체

# LABEL_CNT Check
tab_df['LABEL_CNT'] = new_label_cnt
tab_df[['LABEL_CNT']].value_counts()

LABEL_CNT
3            7126
0            1949
dtype: int64

In [ ]:
# BERTSUM 추출요약 모델에서는 SUMMARY_IDX, SUMMARY_ONEHOT_IDX 정보가 필요하므로 전체 df 저장

tab_df.to_csv('sports_news_for_model_input.csv', index=False)

In [ ]:
# 최종적으로 제출할 column만 추출하여 csv 파일로 저장

final_df = tab_df[['TITLE', 'CONTENT', 'PUBLISH_DT', 'LABEL']].rename(columns={'LABEL':'TRUE_SUMMARY'})
final_df.head()

,TITLE,CONTENT,PUBLISH_DT,TRUE_SUMMARY
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...,2021-07-15,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.\t지...,2021-07-15,유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.\t공...
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\t영국 매체 '스포츠...,2021-07-15,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\t영국 매체 '스포츠...
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다.\t이재성은 ...,2021-07-15,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다.\t1995년...
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다.\t팬들은 만족스럽다는 반응이다....,2021-07-15,바르사는 15일(한국 시간) 공식 홈페이지를 통해 2021/2022시즌에 입을 어웨...


In [ ]:
final_df.to_csv('sports_news_submission_format.csv', index=False)

# Create Data Input for Summarization Models

## MATCHSUM

In [ ]:
original = pd.read_csv('sports_news_for_model_input.csv').dropna().reset_index(drop=True) # 'LABEL' 값이 없는 데이터는 train/inference dataset에서 제외
test_set = original[:2000]
train_set = original[2000:]
test_set.to_csv('sports_news_final_test_set_0324.csv', index=False)
train_set.to_csv('sports_news_final_train_set_0324.csv', index=False)

In [ ]:
test_set.head()

,TITLE,CONTENT,PUBLISH_DT,LABEL,LABEL_CNT,SUMMARY_IDX,SUMMARY_ONEHOT_IDX
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...,2021-07-15,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...,3,"[0, 9, 11]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ..."
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.\t지...,2021-07-15,유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.\t공...,3,"[0, 8, 12]","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\t영국 매체 '스포츠...,2021-07-15,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\t영국 매체 '스포츠...,3,"[0, 1, 8]","[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다.\t이재성은 ...,2021-07-15,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다.\t1995년...,3,"[0, 21, 25]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다.\t팬들은 만족스럽다는 반응이다....,2021-07-15,바르사는 15일(한국 시간) 공식 홈페이지를 통해 2021/2022시즌에 입을 어웨...,3,"[2, 3, 12]","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [ ]:
result = []
df = test_set

for i in tqdm(range(len(df))):
    data_point = df.loc[i]
    doc_lst = list(map(lambda x: x.strip(), data_point['CONTENT'].split('\t')))
    doc_str = '\t'.join(doc_lst)
    summary_lst = list(map(lambda x: x.strip(), data_point['LABEL'].split('\t')))

    for doc in doc_lst:
        binary_label = 0
        
        if doc in summary_lst:
            binary_label = 1

        else:
            binary_label = 0
        
        result.append([doc, doc_str, binary_label])

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
column_names = ['sents', 'docs', 'y']
matchsum_df = pd.DataFrame(result, columns=column_names)
matchsum_df.head()

,sents,docs,y
0,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...,1
1,지난 8일(한국 시간) 잉글랜드와 덴마크는 유로 2020 4강에서 결승 티켓을 두고...,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...,0
2,연장 접전 끝에 잉글랜드가 21로 이겼다.,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...,0
3,하지만 경기 후 논란이 불거졌다.,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...,0
4,11로 팽팽하던 연장 전반 12분 라힘 스털링이 드리블 돌파를 하던 중 요아킴 멜레...,유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크...,0


In [ ]:
matchsum_df[['y']].value_counts()

y
0    30002
1     6000
dtype: int64

In [ ]:
matchsum_df.to_json('sports_news_inference_2000_matchsum_0324.json')